# Laboratorio práctico de WAF

## Configurando protección de Web Application Firewall en un servidor Web

Nota: En una insfraestructura en producción no es común instalarle el WAF dentro del servidor Web. Lo típico es dividir en dos equipos, el servidor web y el WAF. El WAF se configura en modo de proxy reverso de forma que recibe las consultas y las re-envía al servidor web, el cual solo se puede accesar desde el WAF

### Paso #1: Instalar un servidor Web

Por ahora solo vamos a instalar un servidor de páginas normal, e insertar el contenido de la página principal de Wikipedia en ``index.html``

In [ ]:
sudo apt-get install apache2 -y
sudo wget -O /var/www/html/index.html wikipedia.org

Este servidor que instalamos se encuentra desprotegido. En un ambiente de producción de desarrollo se instalarían aplicaciones para contenido dinámico, alguna base de datos, accesos a LDAP, SMTP u otras aplicaciones.

Ahora podría utilizar una herramienta automatizada, un ZAP u OpenVAS para realizar un ataque sobre la aplicación.

Por ello ahora vamos a proceder con la instalación del módulo de ``mod-security`` de Apache el cual hace las funciones de WAF.

También vamos a utilizar el archivo de configuración "recomendada" como base de nuestra configuración.

In [ ]:
sudo apt-get install libapache2-modsecurity -y
apachectl -M | grep --color security
sudo mv -v /etc/modsecurity/modsecurity.conf{-recommended,}
sudo service apache2 reload
sudo ls -l /var/log/apache2/modsec_audit.log

El sistema WAF ya se encuentra activado en el servidor sin embargo todavía no está protegiendo el servidor de forma activa.

Debido a que los sistemas Web son tan complejos, no todas las reglas de un WAF van a funcionar sin cambios para todas las infraestructuras de los usuarios. Por ello el WAF viene configurado en modo ``DetectionOnly``.

Esto se utiliza de forma que en el proceso de pruebas antes de poner el servidor en producción, comenzamos inicialmente con el modo de ``DectectionOnly`` para monitorear errores que pueda presentar el servidor.

En caso de presentar errores, depuramos o deshabilitamos reglas, y una vez que no se presenten errores, procedemos a modificar la opción para se active y comience a bloquear.

Aunque el cambio se puede hacer desde Jupyter-notebook usando herramientas como ``sed``, en este laboratorio se sugiere que utilice el editor de su preferencia para modificar este archivo:

```
sudo vim /etc/modsecurity/modsecurity.conf
```

En el archivo, busque la línea que dice:
```
SecRuleEngine DetectionOnly
```
Y reemplace por:
```
SecRuleEngine On
```
Una vez completada la edición, debe reiniciar el servicio de Web de Apache

In [ ]:
sudo service apache2 reload

Opcionalmente, para equipos con alta carga se puede evaluar cambiar esta opción para evitar procesar encabezados se sitios web según las reglas a utilizar en el WAF. Recuerde que debe reiniciar el servicio de nuevo para aplicar el cambio.
Editando el mismo archivo, busque la línea que dice:
```
SecResponseBodyAccess On
```
Y reemplace por:
```
SecResponseBodyAccess Off
```

## Activando reglas de OWASP

El proyecto de OWASP en su subproyecto de Core Rule Set, permite activar de forma individual reglas del CRS. En nuestro caso vamos a habilitar una regla de protección para ataques de inyección de SQL.
También debe reiniciar el servicio de Web de Apache

In [ ]:
sudo ln -s /usr/share/modsecurity-crs/base_rules/modsecurity_crs_41_sql_injection_attacks.conf .
sudo service apache2 reload